# DSC 575 ASSIGNMENT 4 - SYED NOOR RAZI ALI(2070326)

In [1]:
import numpy as np
import pandas as pd
import math

In [215]:
train_matrix = pd.read_csv("D:/DEPAUL MS DATA SCIENCE/IR/HW4/BBC_News_5 (1)/BBC_News_5_Train.csv", header= None)
train_class_labels = pd.read_csv("D:/DEPAUL MS DATA SCIENCE/IR/HW4/BBC_News_5 (1)/BBC_News_5_Train_Labels.csv")

In [216]:
train_matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,6158,6159,6160,6161,6162,6163,6164,6165,6166,6167
0,1438,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1997,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1810,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,320,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1307,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [3]:
train_class_labels.head()

,sport
0,tech
1,sport
2,business
3,politics
4,entertainment


In [4]:
train_class_labels['sport'].unique()

array(['tech', 'sport', 'business', 'politics', 'entertainment'],
      dtype=object)

In [5]:
test_matrix = pd.read_csv("D:/DEPAUL MS DATA SCIENCE/IR/HW4/BBC_News_5 (1)/BBC_News_5_Test.csv", header= None)
test_class_labels = pd.read_csv("D:/DEPAUL MS DATA SCIENCE/IR/HW4/BBC_News_5 (1)/BBC_News_5_Test_Labels.csv")

# 1. Document Categorization Using Naïve Bayesian Classifier

In [25]:
import numpy as np

def NB_Train(train_matrix, train_class_labels):
    temp = train_class_labels['sport'].value_counts()
    priors = [0] * temp
    for i in range(len(priors)):
        priors[i] = temp[i]
        
    priors = priors/len(train_class_labels)
        
    matrix = train_matrix.copy()
    matrix['label'] = train_class_labels.iloc[:,0]
    matrix = matrix.groupby(['label']).sum()
    matrix['sum'] = matrix.sum(axis=1)
    
    posteriors = {}
    for m in range(len(train_matrix.columns)):
        eacht = [0.0] * temp
        for n in range(len(matrix)):
            eacht[n] = float(((matrix.iloc[n,m])+1)/((matrix.iloc[n,-1]) + len(train_matrix.columns)))
        posteriors[m] = eacht
    
    return posteriors, priors





# A. After training your classifier, provide the prior probabilities associated with each class label.

In [26]:
posteriors, priors = NB_Train(train_matrix, train_class_labels)

In [27]:
label = ['tech','sport','business','politics','entertainment']
for i in range(len(priors)):
    print('P(%s) = %s' %(label[i], priors[i]))

P(tech) = 0.23608768971332209
P(sport) = 0.2270938729623384
P(business) = 0.18324901630129287
P(politics) = 0.18156267566048342
P(entertainment) = 0.17200674536256325


# B. Based on your trained model, provide the posteriors (conditional class probabilities) for each of the following features:  "hollywood", "footbal", "comput", "labour". Note that this is Pr(feature | class) for each class. Here is an example output using the term "tax".

In [134]:
features = ["hollywood", "footbal", "comput", "labour"]
#label = ['tech', 'sport', 'business', 'politics', 'entertainment']

posteriors, priors = NB_Train(train_matrix, train_class_labels)

for feature in features:
    print("Posteriors for the word", feature)
    for i in range(len(label)):
        posterior = posteriors[features.index(feature)][i]
        print("P({} | {}) = {}".format(feature, label[i], posterior))


Posteriors for the word hollywood
P(hollywood | tech) = 0.8530918490910805
P(hollywood | sport) = 0.8432659175264057
P(hollywood | business) = 0.8456343931387807
P(hollywood | politics) = 0.8576602666350835
P(hollywood | entertainment) = 0.8484501680257561
Posteriors for the word footbal
P(footbal | tech) = 1.0876579823219322e-05
P(footbal | sport) = 7.668633245058844e-06
P(footbal | business) = 1.4191711094606677e-05
P(footbal | politics) = 3.820534225300724e-06
P(footbal | entertainment) = 7.050031725142763e-06
Posteriors for the word comput
P(comput | tech) = 1.8127633038698872e-06
P(comput | sport) = 5.112422163372563e-06
P(comput | business) = 7.0958555473033385e-06
P(comput | politics) = 1.910267112650362e-06
P(comput | entertainment) = 2.350010575047588e-06
Posteriors for the word labour
P(labour | tech) = 5.438289911609661e-06
P(labour | sport) = 1.2781055408431407e-05
P(labour | business) = 9.461140729737784e-06
P(labour | politics) = 1.5282136901202895e-05
P(labour | entertai

# C. Provide an output showing actual and predicted class labels for the first 20 document instances (instances 0-19) in the test data along with the log probability values for each test instance. For example, see this sample output for test instances 30-39.

Attempt 1

In [143]:
def NB_Classify(test_case, posteriors, priors):
    # Calculate class probabilities
    num_classes = len(priors)
    class_probs = np.zeros(num_classes)

    for j in range(num_classes):
        log_prob = np.sum(test_case * np.log(posteriors[j])) + np.log(priors[j])
        class_probs[j] = log_prob

    # Determine predicted class label
    predicted_label = np.argmax(class_probs)

    # Map predicted class label to original class label
    label_map = {0: 'business', 1: 'entertainment', 2: 'politics', 3: 'sport', 4: 'tech'}
    predicted_label = label_map[predicted_label]

    return predicted_label, class_probs


In [149]:
for i in range(20):
    test_doc = test_matrix.iloc[i,:]
    actual_label = test_class_labels.iloc[i, 0]
    predicted_label, log_probs = NB_Classify(test_doc, posteriors, priors)
    print("Test Item {}: Actual class = {}, Predicted class = {}, Log probs = {}".format(i, actual_label, predicted_label, log_probs))


Test Item 0: Actual class = tech, Predicted class = business, Log probs = [-1.44355198 -1.48239181 -1.69690931 -1.70615436 -1.76022159]
Test Item 1: Actual class = sport, Predicted class = business, Log probs = [-1.44355198 -1.48239181 -1.69690931 -1.70615436 -1.76022159]
Test Item 2: Actual class = entertainment, Predicted class = business, Log probs = [-1.44355198 -1.48239181 -1.69690931 -1.70615436 -1.76022159]
Test Item 3: Actual class = sport, Predicted class = business, Log probs = [-1.44355198 -1.48239181 -1.69690931 -1.70615436 -1.76022159]
Test Item 4: Actual class = business, Predicted class = business, Log probs = [-1.44355198 -1.48239181 -1.69690931 -1.70615436 -1.76022159]
Test Item 5: Actual class = sport, Predicted class = business, Log probs = [-1.44355198 -1.48239181 -1.69690931 -1.70615436 -1.76022159]
Test Item 6: Actual class = tech, Predicted class = business, Log probs = [-1.44355198 -1.48239181 -1.69690931 -1.70615436 -1.76022159]
Test Item 7: Actual class = busi

Attempt 2

In [318]:
def NB_classify(posteriors, priors, test_matrix, test_class_labels, test_instance):
    class_labels = test_class_labels.iloc[:, 0].unique().tolist()
    difference, total = 0, 0
    intial = priors.copy()
    for q in range(len(priors)):
        intial[q] = math.log(intial[q])
        
    for i in range(test_instance[0], test_instance[1]):
        total += 1
        test = test_matrix.iloc[i,:]
        temp = intial.copy()
        for m in range(len(priors)):
            count = 0
            for j in range(len(test)):
                if test[j] == 0:
                    count += 1
                else:
                    if (posteriors[count][m]**test[j]) != 0:
                        temp[m] += float(math.log(posteriors[count][m]**test[j]))
                    count += 1
            
        #print(temp)
        logtemp = max(temp)
        countindex = 0
        for n in range(len(temp)):
            if temp[n] == logtemp:
                predict = n
        predicted_class = class_labels[predict]
        actual_class = test_class_labels.iloc[i,0]
        if actual_class != predicted_class:
            difference += 1
        
        print('Test Item %s - Predicted Class: %s,- Actual Class: %s Log Prob: %s' %(i, predicted_class, actual_class, logtemp))
    
    return difference, total
    


In [319]:
#output for test instances 0-19
difference, total = NB_classify(posteriors, priors, test_matrix, test_class_labels, [0,20])

Test Item 0 - Predicted Class: sport,- Actual Class: tech Log Prob: -1456.1271164774491
Test Item 1 - Predicted Class: tech,- Actual Class: sport Log Prob: -2727.5884909508454
Test Item 2 - Predicted Class: business,- Actual Class: entertainment Log Prob: -1571.3882900502472
Test Item 3 - Predicted Class: entertainment,- Actual Class: sport Log Prob: -1156.342270258324
Test Item 4 - Predicted Class: business,- Actual Class: business Log Prob: -1545.8282908200335
Test Item 5 - Predicted Class: entertainment,- Actual Class: sport Log Prob: -1222.4109985641992
Test Item 6 - Predicted Class: entertainment,- Actual Class: tech Log Prob: -2315.6015146013597
Test Item 7 - Predicted Class: business,- Actual Class: business Log Prob: -1301.6668431141466
Test Item 8 - Predicted Class: entertainment,- Actual Class: sport Log Prob: -2275.541927756401
Test Item 9 - Predicted Class: business,- Actual Class: sport Log Prob: -888.6763958935245
Test Item 10 - Predicted Class: tech,- Actual Class: busin

# D. Using your evaluation function provide the overall classification accuracy across all test instances.

In [320]:
def classifier_accuracy(difference, total):
    result = (total-difference)/total
    return result

In [321]:
#output for all test instances
end = len(test_matrix)
#difference, total = NB_classify(posteriors, priors, test_matrix, test_class_labels, [0,end])
difference, total = NB_classify(posteriors, priors, test_matrix, test_class_labels, [0, end])


Test Item 0 - Predicted Class: sport,- Actual Class: tech Log Prob: -1456.1271164774491
Test Item 1 - Predicted Class: tech,- Actual Class: sport Log Prob: -2727.5884909508454
Test Item 2 - Predicted Class: business,- Actual Class: entertainment Log Prob: -1571.3882900502472
Test Item 3 - Predicted Class: entertainment,- Actual Class: sport Log Prob: -1156.342270258324
Test Item 4 - Predicted Class: business,- Actual Class: business Log Prob: -1545.8282908200335
Test Item 5 - Predicted Class: entertainment,- Actual Class: sport Log Prob: -1222.4109985641992
Test Item 6 - Predicted Class: entertainment,- Actual Class: tech Log Prob: -2315.6015146013597
Test Item 7 - Predicted Class: business,- Actual Class: business Log Prob: -1301.6668431141466
Test Item 8 - Predicted Class: entertainment,- Actual Class: sport Log Prob: -2275.541927756401
Test Item 9 - Predicted Class: business,- Actual Class: sport Log Prob: -888.6763958935245
Test Item 10 - Predicted Class: tech,- Actual Class: busin

IndexError: single positional indexer is out-of-bounds

In [322]:
classifier_accuracy(difference, total)

0.2

# 2. Document Categorization Using Rocchio Method

In [276]:
def Rocchio_Train(train_matrix, train_class_labels):
    matrix = train_matrix.copy()
    matrix['label'] = train_class_labels.iloc[:,0]
    matrix = matrix.groupby(['label']).sum()
    prototypes = matrix
    
    return prototypes

In [277]:
def CosineSimilarity(test, prototypes):
    resultlist = []

    for i in range(len(prototypes)):
        x, y, xy = 0, 0, 0
        for j in range(len(prototypes.columns)):
            x += test[j] ** 2
            xy += prototypes.iloc[i,j] * test[j]
            y += prototypes.iloc[i,j] ** 2
        temp = xy/(math.sqrt(x) * math.sqrt(y))
        resultlist.append(temp)
    
    return resultlist

In [278]:
def Rocchion_classify(prototypes, test_matrix, test_class_labels, test_instance):
    class_labels = test_class_labels.iloc[:, 0].unique().tolist()
    difference, total = 0, 0
    for i in range(test_instance[0], test_instance[1]):
        total += 1
        test = test_matrix.iloc[i,:]
        resultlist = CosineSimilarity(test, prototypes)
        
        maxSim = max(resultlist)
        predict = resultlist.index(maxSim)
        predicted_class = class_labels[predict]
        actual_class = test_class_labels.iloc[i,0]
       
        if actual_class != predicted_class:
            difference += 1
        
        print('Test Item %s - Predicted Class: %s Actual Class: %s Max Similarity: %s' %(i, predicted_class, actual_class, maxSim))
    
    return difference, total


In [279]:
prototypes = Rocchio_Train(train_matrix, train_class_labels)

In [280]:
#output for test instances 0-19
Rocchion_classify(prototypes, test_matrix, test_class_labels, [0,20])

Test Item 0 - Predicted Class: sport Actual Class: tech Max Similarity: 0.956146306540275
Test Item 1 - Predicted Class: sport Actual Class: sport Max Similarity: 0.9998959720284447
Test Item 2 - Predicted Class: business Actual Class: entertainment Max Similarity: 0.9999596866599461
Test Item 3 - Predicted Class: entertainment Actual Class: sport Max Similarity: 0.9997451292348299
Test Item 4 - Predicted Class: business Actual Class: business Max Similarity: 0.9999663840609617
Test Item 5 - Predicted Class: entertainment Actual Class: sport Max Similarity: 0.9986605932728921
Test Item 6 - Predicted Class: entertainment Actual Class: tech Max Similarity: 0.9999226925830077
Test Item 7 - Predicted Class: business Actual Class: business Max Similarity: 0.9999744787868252
Test Item 8 - Predicted Class: tech Actual Class: sport Max Similarity: 0.9799542601188992
Test Item 9 - Predicted Class: business Actual Class: sport Max Similarity: 0.9999811437845894
Test Item 10 - Predicted Class: bu

(14, 20)

# 2b Using your evaluation function provide the overall classification accuracy across all test instances.

In [281]:
#output for all test instances
end = len(test_matrix)
difference, total = Rocchion_classify(prototypes, test_matrix, test_class_labels, [0,end])

Test Item 0 - Predicted Class: sport Actual Class: tech Max Similarity: 0.956146306540275
Test Item 1 - Predicted Class: sport Actual Class: sport Max Similarity: 0.9998959720284447
Test Item 2 - Predicted Class: business Actual Class: entertainment Max Similarity: 0.9999596866599461
Test Item 3 - Predicted Class: entertainment Actual Class: sport Max Similarity: 0.9997451292348299
Test Item 4 - Predicted Class: business Actual Class: business Max Similarity: 0.9999663840609617
Test Item 5 - Predicted Class: entertainment Actual Class: sport Max Similarity: 0.9986605932728921
Test Item 6 - Predicted Class: entertainment Actual Class: tech Max Similarity: 0.9999226925830077
Test Item 7 - Predicted Class: business Actual Class: business Max Similarity: 0.9999744787868252
Test Item 8 - Predicted Class: tech Actual Class: sport Max Similarity: 0.9799542601188992
Test Item 9 - Predicted Class: business Actual Class: sport Max Similarity: 0.9999811437845894
Test Item 10 - Predicted Class: bu

IndexError: single positional indexer is out-of-bounds

In [285]:
classifier_accuracy(difference, total)

0.9550561797752809

# 3. PageRank

In [238]:
def iteration(initialR, node,alpha):
    R = []
    for i in node:
        if len(node[i]) == 0:
            R.append(alpha/len(node))
        else:
            temp = alpha/len(node)
            for j in node[i]:
                temp += initialR[j] / node[i][j]
            R.append(temp)
              
    norm = np.array(R)*np.array(1/sum(R) )
    return norm

In [242]:
pd.set_option("display.precision", 3)
def PageRank(G, alpha, num_iter):
    initialR, node = {}, {}
    index = []
    for i in G:
        index.append(i)
        initialR[i] = 1/len(G)
        node[i] = {}
        for j in G:
            if i in G[j]:
                node[i][j] = len(G[j])
    
    for time in range(num_iter):
        print('Iteranation:', time)
        norm = iteration(initialR, node,alpha)
        df = pd.DataFrame(norm, columns = ['R'], index = index)
        result = df.transpose()
        display(result)
        initialR = dict(zip(index, norm))

In [243]:
G = {
    'A': ['B','C'],
    'B': ['C','D'],
    'C': ['B'],
    'D': ['C']
}
alpha = 0.25
num_iter = 5
PageRank(G, alpha, num_iter)

Iteranation: 0


,A,B,C,D
R,0.05,0.35,0.45,0.15


Iteranation: 1


,A,B,C,D
R,0.05,0.43,0.33,0.19


Iteranation: 2


,A,B,C,D
R,0.05,0.334,0.394,0.222


Iteranation: 3


,A,B,C,D
R,0.05,0.385,0.381,0.184


Iteranation: 4


,A,B,C,D
R,0.05,0.375,0.371,0.204


In [241]:
G1 = {
    'A': ['C','D','E'],
    'B': ['C','D','E','F'],
    'C': ['D','F'],
    'D': ['C','E'],
    'E': ['A','C'],
    'F': ['A','B','C','D','E']
}
alpha = 0.5
num_iter = 5
PageRank(G1, alpha, num_iter)

Iteranation: 0


,A,B,C,D,E,F
R,0.133,0.078,0.254,0.198,0.198,0.139


Iteranation: 1


,A,B,C,D,E,F
R,0.14,0.074,0.249,0.201,0.183,0.153


Iteranation: 2


,A,B,C,D,E,F
R,0.137,0.076,0.247,0.202,0.187,0.151


Iteranation: 3


,A,B,C,D,E,F
R,0.138,0.076,0.248,0.201,0.186,0.151


Iteranation: 4


,A,B,C,D,E,F
R,0.138,0.076,0.248,0.202,0.186,0.151


# 4. HITS Algorithm (Hubs and Authorities)

In [244]:
def geta(G, h):
    a = {}
    for i in G:
        a[i] = 0
        for j in G:
            if i in G[j]:
                a[i] += h[j]
    return a


def geth(G, a):
    h = {}
    for i in G:
        h[i] = 0
        for j in G[i]:
            h[i] += a[j]
    return h


def normlize(array):
    norm = 0
    for i in array:
        norm += i**2
    return np.sqrt(norm)

In [245]:
def HITS(G, num_iter):
    a, h = {}, {}
    index = list(G.keys())

    for time in range(num_iter):
        if bool(a) is False:
            dictionary = {}
            for key, value in G.items():
                if key not in dictionary:
                    dictionary[key] = 0
                else:
                    dictionary[key] = dictionary[key]
                for i in value:
                    if i in dictionary:
                        dictionary[i] += 1
                    else:
                        dictionary[i] = 1

            sorted_dic = sorted(dictionary.items(), key=lambda kv:(kv[0]))
            a = dict(sorted_dic)
        else:
            a = geta(G,h)
        
       
        h = geth(G,a)
        newa, newh = [],[]
        a = list(a.values())
        h = list(h.values())
        for i, j in zip(a,h):
            newa.append(i/normlize(a))
     
            newh.append(j/normlize(h))
     
        
        a, h = {}, {}
        a = dict(zip(index, newa))
        h = dict(zip(index, newh))

        newtable = {'Authority': list(a.values()), 'Hub': list(h.values())}
        df = pd.DataFrame(newtable, index = index)
        result = df.transpose()
        display(result)

In [247]:
HITS(G1, 5)

,A,B,C,D,E,F
Authority,0.246,0.123,0.615,0.492,0.492,0.246
Hub,0.455,0.525,0.210,0.315,0.245,0.560


,A,B,C,D,E,F
Authority,0.228,0.159,0.596,0.497,0.526,0.209
Hub,0.459,0.519,0.200,0.318,0.234,0.569


,A,B,C,D,E,F
Authority,0.228,0.161,0.596,0.496,0.529,0.204
Hub,0.460,0.518,0.198,0.319,0.234,0.570


,A,B,C,D,E,F
Authority,0.228,0.162,0.596,0.495,0.529,0.203
Hub,0.460,0.517,0.198,0.319,0.234,0.570


,A,B,C,D,E,F
Authority,0.228,0.162,0.596,0.495,0.530,0.203
Hub,0.460,0.517,0.198,0.319,0.234,0.570
